In [1]:
!pip install deepfilternet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.2/113.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.8/568.8 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.2
    Uninstalling packaging-24.2:
      Successfully uninstalled packaging-24.2
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 require

In [2]:
from google.colab import files

audio_file = files.upload()


Saving test (1).mp3 to test (1).mp3


In [8]:
audio_file = list(audio_file.keys())[0]

In [4]:
import torch

In [5]:
from df.enhance import enhance, init_df, load_audio, save_audio

model, df_state, _ = init_df()

device = "cuda" if torch.cuda.is_available() else "cpu"
compute_type = "float16" if device == "cuda" else "int8"

print(f"✅ Using device: {device}")


2025-07-13 04:37:40 | INFO     | DF | Loading model settings of DeepFilterNet3
2025-07-13 04:37:40 | INFO     | DF | Using DeepFilterNet3 model at /root/.cache/DeepFilterNet/DeepFilterNet3
2025-07-13 04:37:40 | INFO     | DF | Initializing model `deepfilternet3`
2025-07-13 04:37:40 | INFO     | DF | Found checkpoint /root/.cache/DeepFilterNet/DeepFilterNet3/checkpoints/model_120.ckpt.best with epoch 120
2025-07-13 04:37:40 | INFO     | DF | Running on device cuda:0
2025-07-13 04:37:40 | INFO     | DF | Model loaded
✅ Using device: cuda


In [10]:
import os

In [13]:
audio, _ = load_audio(audio_file, sr=df_state.sr())
denoised_dir = "denoised"
os.mkdir(denoised_dir)
enhanced_path = os.path.join(denoised_dir, "enhanced.wav")
enhanced = enhance(model, df_state, audio)
save_audio(enhanced_path, enhanced, df_state.sr())

In [15]:
from IPython.display import Audio

Audio('denoised/enhanced.wav')

